In [1]:
import robotic as ry
import numpy as np
import random
from bridgebuild_env import create_n_boxes

# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip
import robot_execution as robex

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Task Env: Bridge Builder

In [2]:
C = ry.Config()
C.addFile(ry.raiPath("scenarios/pandaSingle.g"))
n_boxes = 3
keep_dist_between = ["box1", "box2", "box3"]
C = create_n_boxes(C=C, N=n_boxes, position="fixed")

# remove so collision with wrist camera is ignored
C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper"
palm = "l_palm"
box1 = "box1"
box2 = "box2"
box3 = "box3"
table = "table"
boxSize = C.frame(box1).getSize()

C.view()

0

In [24]:
C.setJointState(qHome)
C.view_raise()
boxes_pos = [
    C.frame(box1).getPosition(),
    C.frame(box2).getPosition(),
    C.frame(box3).getPosition(),
]
boxes = [box1, box2, box3]
last_box_memory = []
n_success = 0
while n_success <= 2:
    C.setJointState(qHome)
    box = np.random.choice(boxes)
    qStart = C.getJointState()
    graspDirection = random.choice(["x", "y"])  # random.choice(['y', 'z']) #'x' not possible: box too large
    placeDirection = "z"  # random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
    info = f"placement {n_success}: grasp {graspDirection} place {placeDirection}"
    print("===", info)

    M = manip.ManipulationModelling(C, info, helpers=[gripper])
    M.setup_pick_and_place_waypoints(
        gripper, box, homing_scale=1.8e-1, accumulated_collisions=True
    )
    M.grasp_box(1.0, gripper, box, palm, graspDirection)
    print("place direction:", placeDirection)

    if n_success > 1:

        print(f"3rd Block: ({box})")

        dir_vec = (
            C.getFrame(last_box_memory[-2]).getPosition()
            - C.getFrame(last_box_memory[-1]).getPosition()
        )
        pos2 = C.getFrame(last_box_memory[-2]).getPosition()
        pos1 = C.getFrame(last_box_memory[-1]).getPosition()
        rel_pos = (pos1 + pos2) / 2 + np.array([0, 0, 0.080])
        dir_vec /= np.linalg.norm(dir_vec)

        M.komo.addObjective([2.0], ry.FS.position, [box], ry.OT.eq, [1e1], rel_pos)
        M.komo.addObjective([2.0], ry.FS.vectorZ, [box], ry.OT.eq, [1e1], dir_vec)
        # M.keep_dist_pairwise([], keep_dist_between, margin=0.08)

        print("Grasp Direction:", graspDirection)
        # M.komo.addObjective([2.], ry.FS.vectorZ, [gripper], ry.OT.eq, [1e1], [0, 0, 1])

        if graspDirection == "x":
            M.komo.addObjective(
                [2.0], ry.FS.scalarProductXZ, [box, table], ry.OT.eq, [1e1], [0.0]
            )
        elif graspDirection == "y":
            M.komo.addObjective(
                [2.0], ry.FS.scalarProductYZ, [box, table], ry.OT.eq, [1e1], [0.0]
            )

    if n_success == 1:
        print(f"2nd Block: ({box})")
        M.place_box(2.0, box, table, palm, placeDirection)

        M.komo.addObjective(
            [2.0],
            ry.FS.negDistance,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1e1],
            [-0.04],
        )
        M.komo.addObjective(
            [2.0],
            ry.FS.scalarProductXY,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1e1],
            [0],
        )
        M.komo.addObjective(
            [2.0],
            ry.FS.positionRel,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1.0],
            [0.0, 0.0, 0.1],
        )
        #M.keep_dist_pairwise([], keep_dist_between, margin=0.08)

    #M.keep_distance([], palm, table)
    # keep distance between boxes when placed defined by margin
    # x, y, z, smooth - table size: array([2.5 , 2.5 , 0.1 , 0.02])
    # place inital block as chosen by np.random.choice at a random location
    if n_success == 0:
        print(f"1st Block: ({box})")
        M.place_box(2.0, box, table, palm, placeDirection)
        initial_pos = [np.random.uniform(0.0, 0.1), np.random.uniform(0.0, 0.1)]
        # initial_pos= [0.020433060890705013, 0.08433819136314634]
        print("Trying xy pos: ", initial_pos)
        M.target_relative_xy_position(2.0, box, table, initial_pos)
        M.keep_dist_pairwise([], keep_dist_between, margin=0.08)
    
    ways = M.solve()

    if not M.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M :("
        )
        continue
    print("Submotion 0")

    M2 = M.sub_motion(0)

    #M2.keep_distance([0.3, 0.7], palm, box)

    M2.retract([0.0, 0.2], gripper, dist=0.13)
    M2.approach([0.8, 1.0], gripper)
    M2.solve()
    if not M2.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M2 :("
        )
        continue

    M3 = M.sub_motion(1)

    #M3.keep_distance([], table, box)
    
    if n_success > 1:
        M3.retract([0.0, 0.2], gripper, dist=0.13)
        M3.approach([0.8, 1.0], gripper)


    M3.solve()
    if not M3.ret.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M3 :("
        )
        continue

    M2.play(C)
    C.attach(gripper, box)
    M3.play(C)
    C.attach(table, box)
    # update memory
    last_box_memory.append(box)
    # remove box that was previously used
    boxes.remove(box)
    # update positions
    boxes_pos = [
        C.frame(box1).getPosition(),
        C.frame(box2).getPosition(),
        C.frame(box3).getPosition(),
    ]
    n_success += 1

del M

=== placement 0: grasp y place z
place direction: z
1st Block: (box3)
Trying xy pos:  [0.0718337507698331, 0.026018038374756892]
  -- infeasible:placement 0: grasp y place z
     { time: 0.100613, evals: 201, done: 1, feasible: 0, sos: 0.131871, f: 0, ineq: 0.0373197, eq: 0.575488 }
Failed placing block number 1 with grasp direction y at M :(
=== placement 0: grasp x place z
place direction: z
1st Block: (box2)
Trying xy pos:  [0.08275407234044804, 0.030327129919519382]
  -- feasible:placement 0: grasp x place z
     { time: 0.065125, evals: 134, done: 1, feasible: 1, sos: 0.215904, f: 0, ineq: 0.00233847, eq: 0.286784 }
Submotion 0
  -- feasible:sub_motion_0--placement 0: grasp x place z
     { time: 0.044885, evals: 22, done: 1, feasible: 1, sos: 2.98437, f: 0, ineq: 2.51357e-07, eq: 0.320034 }
  -- feasible:sub_motion_1--placement 0: grasp x place z
     { time: 0.236935, evals: 98, done: 1, feasible: 1, sos: 0.142281, f: 0, ineq: 0, eq: 0.328114 }
=== placement 1: grasp y place z
p

KeyboardInterrupt: 

### On real robot: BotOp

In [1]:
import robotic as ry
import numpy as np
from bridgebuild_env import create_n_boxes
import robot_execution as robex
# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip
import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
C = ry.Config()
C.addFile(ry.raiPath("scenarios/pandaSingle.g"))
n_boxes = 3
keep_dist_between = ["box1", "box2", "box3"]
C = create_n_boxes(C=C, N=n_boxes, position="fixed")

# remove so collision with wrist camera is ignored
C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper"
palm = "l_palm"
box1 = "box1"
box2 = "box2"
box3 = "box3"
table = "table"
boxSize = C.frame(box1).getSize()

C.view()

0

In [ ]:
C.view()
bot = robex.Robot(C, on_real=True)
bot.goHome(C)

: 

In [4]:
C.setJointState(qHome)
C.view_raise()
boxes_pos = [
    C.frame(box1).getPosition(),
    C.frame(box2).getPosition(),
    C.frame(box3).getPosition(),
]
boxes = [box1, box2, box3]
last_box_memory = []
paths = []
vis = 1
n_success = 0
boxz = [box1, box3, box2]
while n_success <= 2:
    box = np.random.choice(boxes)
    qStart = C.getJointState()
    graspDirection = random.choice(["x", "y"])  # z grasp removed
    placeDirection = "z"  # random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
    info = f"placement {n_success}: grasp {graspDirection} place {placeDirection}"
    print("===", info)

    M = manip.ManipulationModelling(C, info, helpers=[gripper])
    M.setup_pick_and_place_waypoints(
        gripper, box, homing_scale=1.8e-1, accumulated_collisions=True
    )
    M.grasp_box(1.0, gripper, box, palm, graspDirection)
    print("place direction:", placeDirection)

    if n_success > 1:
        print(f"3rd Block: ({box})")
        dir_vec = (
            C.getFrame(last_box_memory[-2]).getPosition()
            - C.getFrame(last_box_memory[-1]).getPosition()
        )
        pos2 = C.getFrame(last_box_memory[-2]).getPosition()
        pos1 = C.getFrame(last_box_memory[-1]).getPosition()
        rel_pos = (pos1 + pos2) / 2 + np.array([0, 0, 0.08])
        dir_vec /= np.linalg.norm(dir_vec)

        M.komo.addObjective([2.0], ry.FS.position, [box], ry.OT.eq, [1e1], rel_pos)
        M.komo.addObjective([2.0], ry.FS.vectorZ, [box], ry.OT.eq, [1e1], dir_vec)
        #M.keep_dist_pairwise([], keep_dist_between, margin=0.08)

        print("Grasp Direction:", graspDirection)
        # M.komo.addObjective([2.], ry.FS.vectorZ, [gripper], ry.OT.eq, [1e1], [0, 0, 1])

        if graspDirection == "x":
            M.komo.addObjective(
                [2.0], ry.FS.scalarProductXZ, [box, table], ry.OT.eq, [1e1], [0.0]
            )
        elif graspDirection == "y":
            M.komo.addObjective(
                [2.0], ry.FS.scalarProductYZ, [box, table], ry.OT.eq, [1e1], [0.0]
            )

    if n_success == 1:
        print(f"2nd Block: ({box})")
        M.place_box(2.0, box, table, palm, placeDirection)
        M.komo.addObjective(
            [2.0],
            ry.FS.negDistance,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1e1],
            [-0.04],
        )
        M.komo.addObjective(
            [2.0],
            ry.FS.scalarProductXY,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1e1],
            [0],
        )
        M.komo.addObjective(
            [2.0],
            ry.FS.positionRel,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1.0],
            [0.0, 0.0, 0.1],
        )
        #M.keep_dist_pairwise([], keep_dist_between, margin=0.08)

    M.keep_distance([], palm, table)
    # keep distance between boxes when placed defined by margin
    # x, y, z, smooth - table size: array([2.5 , 2.5 , 0.1 , 0.02])
    # place inital block as chosen by np.random.choice at a random location
    if n_success == 0:
        print(f"1st Block: ({box})")
        M.place_box(2.0, box, table, palm, placeDirection)
        initial_pos = [np.random.uniform(0.0, 0.1), np.random.uniform(0.0, 0.1)]
        # initial_pos= [0.020433060890705013, 0.08433819136314634]
        print("Trying xy pos: ", initial_pos)
        M.target_relative_xy_position(2.0, box, table, initial_pos)
        #M.keep_dist_pairwise([], keep_dist_between, margin=0.08)

    ways = M.solve()

    if not M.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M :("
        )
        continue
    print("Submotion 0")
    M2 = M.sub_motion(0)

    M2.keep_distance([0.3, 0.7], palm, box)

    M2.retract([0.0, 0.2], gripper, dist=0.15)
    M2.approach([0.8, 1.0], gripper)
    paths.append(M2.solve())

    if not M2.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M2 :("
        )
        continue

    M3 = M.sub_motion(1)

    M3.keep_distance([], table, box)

    if n_success > 1:
        M3.retract([0.0, 0.2], gripper, dist=0.15)
        M3.approach([0.8, 1.0], gripper)

    paths.append(M3.solve())
    if not M3.ret.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M3 :("
        )
        continue

    bot.execute_path_blocking(C, path=paths[-2])
    C.attach(gripper, box)
    bot.grasp(C)

    bot.execute_path_blocking(C, path=paths[-1])
    C.attach(table, box)
    bot.release(C)
    # update memory
    last_box_memory.append(box)
    # remove box that was previously used
    boxes.remove(box)
    # update positions
    boxes_pos = [
        C.frame(box1).getPosition(),
        C.frame(box2).getPosition(),
        C.frame(box3).getPosition(),
    ]
    n_success += 1

del M

=== placement 0: grasp x place z
place direction: z
1st Block: (box2)
Trying xy pos:  [0.05081675454154131, 0.0009059812097997]
  -- feasible:placement 0: grasp x place z
     { time: 0.051205, evals: 111, done: 1, feasible: 1, sos: 0.207519, f: 0, ineq: 0.0132057, eq: 0.393276 }
Submotion 0
  -- feasible:sub_motion_0--placement 0: grasp x place z
     { time: 0.037942, evals: 21, done: 1, feasible: 1, sos: 3.61862, f: 0, ineq: 1.49874e-07, eq: 0.322577 }
  -- feasible:sub_motion_1--placement 0: grasp x place z
     { time: 0.428904, evals: 179, done: 1, feasible: 1, sos: 0.188491, f: 0, ineq: 0, eq: 0.342142 }
=== placement 1: grasp y place z
place direction: z
2nd Block: (box3)
  -- infeasible:placement 1: grasp y place z
     { time: 0.094855, evals: 200, done: 1, feasible: 0, sos: 0.315035, f: 0, ineq: 0.0837747, eq: 0.680344 }
Failed placing block number 2 with grasp direction y at M :(
=== placement 1: grasp y place z
place direction: z
2nd Block: (box3)
  -- infeasible:placement